In [1]:
import sqlite3
import pandas as pd
import re

## Download Data

Outline:


1) Need to download SQLite browser (???)
Download sqlite from https://www.sqlite.org/download.html
or sqlite browser from https://sqlitebrowser.org/dl/
2) Need sqlite3 module and probably pandas

3) Virtually run jupyter notebook? L asking Robin and Eric about best ways to proceed

3) Jupyter notebook: 2 versions (answers and questions)

4) Figure out the best way to import data from pandas dataframes.

5) Introduction to databases in general: excel chart to dataframes to 

6) Creating the database- loading in data, constructing tables - L and D need to figure it out

7) After create database, start playing around with queries

8) Ask Mike about setting up sql on our own computers

In [2]:
# Data sets available at https://catalog.data.gov/dataset
data = pd.read_csv('NCHS_-_Leading_Causes_of_Death__United_States.csv',',')


In [3]:
# Look at data
data.head()

,Year,113 Cause Name,Cause Name,State,Deaths,Age-adjusted Death Rate
0,2012,"Nephritis, nephrotic syndrome and nephrosis (N...",Kidney disease,Vermont,21,2.6
1,2016,"Nephritis, nephrotic syndrome and nephrosis (N...",Kidney disease,Vermont,30,3.7
2,2013,"Nephritis, nephrotic syndrome and nephrosis (N...",Kidney disease,Vermont,30,3.8
3,2000,"Intentional self-harm (suicide) (*U03,X60-X84,...",Suicide,District of Columbia,23,3.8
4,2014,"Nephritis, nephrotic syndrome and nephrosis (N...",Kidney disease,Arizona,325,4.1


In [4]:
# What are the column names?
print([col for col in data.columns])

['Year', '113 Cause Name', 'Cause Name', 'State', 'Deaths', 'Age-adjusted Death Rate']


In [5]:
# What are unique causes of death
print(set(data['Cause Name'].values))

{'Suicide', 'Influenza and pneumonia', 'Unintentional injuries', 'Kidney disease', 'Heart disease', 'Stroke', 'All causes', "Alzheimer's disease", 'CLRD', 'Diabetes', 'Cancer'}


In [6]:
#Number of characters in cause
print([len(list(cause)) for cause in set(data['Cause Name'].values)])

[7, 23, 22, 14, 13, 6, 10, 19, 4, 8, 6]


In [25]:
def makeDFsFromColumns(L,dataframe,dfDict):
    for item in L:
        #Rename item to avoid spaces
        item_new_name = re.sub(' ', '_', item)
        item_new_name = re.sub('113', '_113', item_new_name)
        
        #Take unique items, put into dataframe
        data_unique = dataframe[item].unique()
        unique_df = pd.DataFrame(data_unique, columns = [item_new_name])
        
        #Save as csv file with new file
        dfDict[item_new_name] = unique_df

# Make files of unique values for all data except "Deaths" and "Age-adjusted Death Rate"
L = [col for col in data.columns if col not in ["Deaths", "Age-adjusted Death Rate"]]

# Put unique dataframes into a dictionary
dfDict = {}
makeDFsFromColumns(L,data,dfDict)

main_data_frame = pd.DataFrame(data, columns = ["Year", "Cause Name", "State", "Deaths", "Age-adjusted Death Rate"])
dfDict['Deaths'] = main_data_frame
print(dfDict.keys())


dict_keys(['Year', '_113_Cause_Name', 'Cause_Name', 'State', 'main_table'])


In [8]:
# Make single table as an example
conn = sqlite3.connect("leading_cases_of_death.sqlite")
cur = conn.cursor()
sql_statement1 = ("DROP TABLE IF EXISTS Cause")
#sql_statement2 = ''' CREATE TABLE Cause (
                    #CauseNumber INTEGER PRIMARY KEY,
                    #Cause VARCHAR(23) NOT NULL
                    #);'''
cur.execute(sql_statement1)
#cur.execute(sql_statement2)

#cur.executemany("INSERT INTO Cause (Cause) VALUES (?)", dfDict['Cause_Name'].values)



In [9]:
# What does the table look like?
#df = pd.read_sql_query("SELECT * FROM Cause", conn)
#conn.close()


In [26]:
# Automate
# Make files of unique values for all data except "Deaths" and "Age-adjusted Death Rate"
L = [col for col in data.columns if col not in ["Deaths", "Age-adjusted Death Rate"]]
L = [re.sub(' ', '_', item) for item in L]
L = [re.sub('113', '_113', item) for item in L]

def loadSmallTables(conn,L,dfDict):
    for variable in L:
        df = dfDict[variable]
        items = [item[0] for item in df.values]
        maxLength = max([len(list(str(item))) for item in items])
        sql_statement1 = ("DROP TABLE IF EXISTS {0}".format(variable))
        cur.execute(sql_statement1)
        f_list = [variable, variable + 'ID', 'VARCHAR', maxLength]
        if variable == 'Year':
            f_list[2] = 'INTEGER'
            f_list[3] = '' 
        sql_statement2 = '''CREATE TABLE {0} (
                            {1} INTEGER PRIMARY KEY,
                            {0} {2}{3} NOT NULL
                            );'''.format(f_list[0], f_list[1], f_list[2], f_list[3])
        cur.execute(sql_statement2)
        sql_statement3 = "INSERT INTO {0} ({0}) VALUES (?);".format(variable)
        cur.executemany(sql_statement3, df.values)

loadSmallTables(conn,L,dfDict)

# Load main table
sql_statement1 = ("DROP TABLE IF EXISTS Deaths")
sql_statement2 = '''CREATE TABLE Deaths (
                    Deaths INTEGER,
                    Age-adjusted_Death_Rate FLOAT,
                    Cause_Name_ID INTEGER,
                    Year_ID INTEGER,
                    State_ID INTEGER,
                    FOREIGN KEY (Cause_Name_ID) REFERENCES Cause_Name(Cause_Name_ID)
                    
                    
                    )'''


conn.close()

INSERT INTO Year (Year) VALUES (?);
[2012, 2016, 2013, 2000, 2014, 2009, 2011, 2015, 2001, 1999, 2006, 2002, 2003, 2008, 2005, 2004, 2007, 2010]
INSERT INTO _113_Cause_Name (_113_Cause_Name) VALUES (?);
['Nephritis, nephrotic syndrome and nephrosis (N00-N07,N17-N19,N25-N27)', 'Intentional self-harm (suicide) (*U03,X60-X84,Y87.0)', "Alzheimer's disease (G30)", 'Influenza and pneumonia (J09-J18)', 'Diabetes mellitus (E10-E14)', 'Chronic lower respiratory diseases (J40-J47)', 'Accidents (unintentional injuries) (V01-X59,Y85-Y86)', 'Cerebrovascular diseases (I60-I69)', 'Diseases of heart (I00-I09,I11,I13,I20-I51)', 'Malignant neoplasms (C00-C97)', 'All Causes']
INSERT INTO Cause_Name (Cause_Name) VALUES (?);
['Kidney disease', 'Suicide', "Alzheimer's disease", 'Influenza and pneumonia', 'Diabetes', 'CLRD', 'Unintentional injuries', 'Stroke', 'Heart disease', 'Cancer', 'All causes']
INSERT INTO State (State) VALUES (?);
['Vermont', 'District of Columbia', 'Arizona', 'South Dakota', 'Washing

In [22]:
[print(key) for key in dfDict.keys()]

Year
_113 Cause Name
Cause Name
State
main_table


[None, None, None, None, None]

In [19]:
print(L)


['Year', '_113_Cause_Name', 'Cause_Name', 'State']
